In [1]:
!pip install mysql

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import mysql.connector
import pandas as pd
from openpyxl import load_workbook

# Configuration de la connexion à MySQL
config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'valide_ton_semestre',
    'raise_on_warnings': True
}

try:
    # Établir la connexion à MySQL
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    print("Connexion à la base de données réussie")

    # Chemin du fichier Excel
    excel_file = 'SD-MCC-2024-25.xlsx'
    
    # Charger le fichier Excel
    wb = load_workbook(filename=excel_file)
    
    # Étape 1: Insérer les semestres
    if 'Semestres' in wb.sheetnames:
        sheet = wb['Semestres']
        print("Importation des semestres...")
        
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if row[0]:  # Vérifier que la ligne n'est pas vide
                cursor.execute(
                    "INSERT INTO semestre_choix (nom_semestre, description) VALUES (%s, %s)",
                    (row[0], row[1] if len(row) > 1 else None)
                )
        conn.commit()
        print(f"{cursor.rowcount} semestres importés")

    # Étape 2: Insérer les UE
    if 'UE' in wb.sheetnames:
        sheet = wb['UE']
        print("Importation des UE...")
        
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if row[0]:
                cursor.execute(
                    "INSERT INTO ue (Nom_UE) VALUES (%s)",
                    (row[0],)
                )
        conn.commit()
        print(f"{cursor.rowcount} UE importées")
    
    # Étape 3: Créer un mapping des UE (nom -> id)
    cursor.execute("SELECT id, Nom_UE FROM ue")
    ue_mapping = {nom: id for id, nom in cursor.fetchall()}
    
    # Étape 4: Insérer les matières
    if 'Matieres' in wb.sheetnames:
        sheet = wb['Matieres']
        print("Importation des matières...")
        
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if row[0] and row[1] and row[2]:  # Nom, Coefficient, UE
                if row[2] in ue_mapping:
                    ue_id = ue_mapping[row[2]]
                    cursor.execute(
                        "INSERT INTO matieres (Nom_Matiere, Coefficient, UE_id) VALUES (%s, %s, %s)",
                        (row[0], row[1], ue_id)
                    )
                else:
                    print(f"ATTENTION: UE '{row[2]}' non trouvée pour la matière '{row[0]}'")
        conn.commit()
        print(f"{cursor.rowcount} matières importées")

    # Étape 5: Créer un mapping des semestres (nom -> id)
    cursor.execute("SELECT id, nom_semestre FROM semestre_choix")
    semestre_mapping = {nom: id for id, nom in cursor.fetchall()}
    
    # Étape 6: Insérer les UE par semestre
    if 'UE_par_semestre' in wb.sheetnames:
        sheet = wb['UE_par_semestre']
        print("Importation des UE par semestre...")
        
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if len(row) >= 4 and row[0] and row[1] and row[2] and row[3]:
                # semester_id, UE, name_field, coefficient_field
                cursor.execute(
                    "INSERT INTO ues_choix (semester_id, UE, name_field, coefficient_field) VALUES (%s, %s, %s, %s)",
                    (row[0], row[1], row[2], row[3])
                )
        conn.commit()
        print(f"{cursor.rowcount} associations UE-semestre importées")

    print("Importation terminée avec succès!")

except mysql.connector.Error as err:
    print(f"Erreur MySQL: {err}")

except Exception as e:
    print(f"Erreur: {e}")

finally:
    if 'conn' in locals() and conn.is_connected():
        cursor.close()
        conn.close()
        print("Connexion fermée")


ModuleNotFoundError: No module named 'mysql'